<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Visualización de Datos</p>
<p style="margin: 0; text-align:right;">Máster Universitario de Ciencia de datos </p>
<p style="margin: 0; text-align:right;">Universitat Oberta de Catalunya</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Autor: Jorge Alonso Hernández</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Práctica 2: Creación de la visualización y entrega del proyecto

In [1]:
#Carga de librerías
import pandas as pd
import numpy as np 
import itertools

from datetime import datetime

In [2]:
data_covid = pd.read_csv("datasets/covid_canarias.csv")
municipios = pd.read_csv("datasets/municipios.csv")
islas = pd.read_csv("datasets/islas.csv")

In [3]:
data_covid.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
0,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
1,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
2,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
3,14/03/2022,TENERIFE,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
4,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife


In [8]:
#Obtenenmos los nombres de los municipios en el documento de casos de covid
municipios_ori = pd.DataFrame(pd.unique(data_covid['municipio']))
municipios_ori.columns = ['municipio']

#Comprobamos los nombres de los municipios
df_destino = municipios.loc[~municipios['municipio'].isin(pd.unique(data_covid['municipio']))]
df_destino = pd.DataFrame(df_destino['municipio'])
df_origen = municipios_ori.loc[~municipios_ori['municipio'].isin(municipios['municipio'])]

df_origen = df_origen.sort_values(by='municipio')
df_destino = df_destino.sort_values(by='municipio')

df_origen = df_origen.reset_index()
df_destino = df_destino.reset_index()

In [9]:
print(df_origen)

   index              municipio
0     45               El Pinar
1     81           Fuencaliente
2     70      Icod de Los Vinos
3      5              La Laguna
4     24      Puerto de La Cruz
5     87        SIN ESPECIFICAR
6     32  San juan de La Rambla
7     29    Santa María de Guía
8     67            Valsequillo
9     69               Vilaflor


In [10]:
print(df_destino)

   index                            municipio
0     87                El Pinar de El Hierro
1     47             Fuencaliente de La Palma
2     55                    Icod de los Vinos
3     61                    Puerto de la Cruz
4     56           San Cristóbal de La Laguna
5     67                San Juan de la Rambla
6     22  Santa María de Guía de Gran Canaria
7     30          Valsequillo de Gran Canaria
8     85                   Vilaflor de Chasna


In [11]:
#Eliminamos el valor SIN ESPECIFICAR pues es correcto que estos valores no se visualicen en el mapa 
df_origen = df_origen.drop(5).reset_index()

In [12]:
print(df_origen)

   level_0  index              municipio
0        0     45               El Pinar
1        1     81           Fuencaliente
2        2     70      Icod de Los Vinos
3        3      5              La Laguna
4        4     24      Puerto de La Cruz
5        6     32  San juan de La Rambla
6        7     29    Santa María de Guía
7        8     67            Valsequillo
8        9     69               Vilaflor


In [13]:
for i in range(0,len(df_origen)):
    data_covid = data_covid.replace(df_origen['municipio'][i], df_destino['municipio'][i])


In [14]:
print(data_covid[data_covid['municipio'] == 'San Cristóbal de La Laguna'])

       fecha_datos       isla                   municipio    sexo grupo_edad  \
361     14/03/2022   TENERIFE  San Cristóbal de La Laguna  Hombre    30 - 39   
1099    14/03/2022   TENERIFE  San Cristóbal de La Laguna  Hombre    30 - 39   
1100    14/03/2022   TENERIFE  San Cristóbal de La Laguna  Hombre    40 - 49   
1101    14/03/2022   TENERIFE  San Cristóbal de La Laguna  Hombre      0 - 9   
1102    14/03/2022   TENERIFE  San Cristóbal de La Laguna  Hombre    50 - 59   
...            ...        ...                         ...     ...        ...   
563957  14/03/2022   TENERIFE  San Cristóbal de La Laguna   Mujer    50 - 59   
563959  14/03/2022   TENERIFE  San Cristóbal de La Laguna  Hombre    40 - 49   
565059  14/03/2022   TENERIFE  San Cristóbal de La Laguna   Mujer    30 - 39   
565390  14/03/2022  EL HIERRO  San Cristóbal de La Laguna  Hombre    30 - 39   
565767  14/03/2022  LANZAROTE  San Cristóbal de La Laguna  Hombre    60 - 69   

        fecha_caso fecha_fallecido fech

In [15]:
asignacion = data_covid[data_covid['identificacion_caso'] == 'Asignación']
residencia = data_covid[data_covid['identificacion_caso'] == 'Residencia']

In [16]:
df_municipios = pd.DataFrame(municipios['municipio'])
df_islas = pd.DataFrame(islas['isla'])
df_provincias = pd.DataFrame(pd.unique(islas['provincia']))
df_provincias.columns = ['provincia']

In [17]:
capitales = ['Santa Cruz de Tenerife', 'Santa Cruz de La Palma', 'San Sebastián de La Gomera', 'Valverde', 'Las Palmas de Gran Canaria', 'Puerto del Rosario', 'Arrecife']
df_capitales = municipios.loc[municipios['municipio'].isin(capitales)]
print(df_capitales)

    geocode geocode_provincia geocode_isla                   municipio  \
3     35004             ES701        ES708                    Arrecife   
15    35016             ES701        ES705  Las Palmas de Gran Canaria   
16    35017             ES701        ES704          Puerto del Rosario   
69    38036             ES702        ES706  San Sebastián de La Gomera   
70    38037             ES702        ES707      Santa Cruz de La Palma   
71    38038             ES702        ES709      Santa Cruz de Tenerife   
81    38048             ES702        ES703                    Valverde   

    sueprficie  perimetro   longitud    latitud  longitud_capital  \
3      2427.54      36641 -13.551117  28.978878        -13.546719   
15    10331.60     108650 -15.458582  28.102743        -15.428533   
16    29077.10      92564 -13.971636  28.499263        -13.859694   
69    11313.40      59100 -17.167078  28.094869        -17.109706   
70     4341.21      31883 -17.798783  28.705235        -17.764

In [18]:
df_capitales = df_capitales[['municipio', 'longitud_capital','latitud_capital']]

In [19]:
def conversion(df, tipo_agrupamiento):
    df_result = pd.DataFrame()
    df_result_accumul = pd.DataFrame()
        
    if tipo_agrupamiento == 'municipios':
        df_agrupamientos = df_municipios
        label = 'municipio'
    elif tipo_agrupamiento == 'islas':
        df_agrupamientos = df_islas
        label = 'isla'
    elif tipo_agrupamiento == 'provincias':
        df_agrupamientos = df_provincias
        label = 'provincia'
    else:
        return None
    
    for index, row in df_agrupamientos.iterrows():
        aux = df[df[label] == row[label]]
        aux = aux.groupby(['fecha_caso',label]).size().reset_index()
        aux.columns=['fecha', label, row[label]]
        aux = aux[['fecha', row[label]]]
        aux['fecha'] = pd.to_datetime(aux['fecha'], dayfirst=True,format='%d/%m/%Y', exact=True)
        aux['fecha'] = aux['fecha'].dt.strftime('%Y/%m/%d')
        
        aux = aux.sort_values(by='fecha')
        aux = aux.set_index('fecha')
        aux_acc = aux.cumsum()
        aux = aux.transpose()
        aux_acc = aux_acc.transpose()
        
        df_result = pd.concat([df_result, aux], axis = 0)
        df_result_accumul = pd.concat([df_result_accumul, aux_acc], axis = 0)
    
    
    ## Ordenamos por fecha y cumplimentamos los NaN con el valor anterior
    df_result_accumul = df_result_accumul.transpose()
    df_result_accumul = df_result_accumul.sort_values(by=['fecha'])
    df_result_accumul[0:1] = df_result_accumul[0:1].fillna(0)   
    df_result_accumul = df_result_accumul.fillna(method="pad")
    df_result_accumul = df_result_accumul.transpose()
    
    #Ordenamos por fecha y cumplimentamos los NaN por 0
    df_result = df_result.replace(np.nan, 0)
    df_result = df_result.transpose()
    df_result = df_result.sort_values(by=['fecha'])
    df_result = df_result.transpose()
    
    df_result = df_result.reset_index()
    df_result_accumul = df_result_accumul.reset_index()
    
    return df_result, df_result_accumul

In [20]:
data_municipios, data_municipios_accumul =  conversion(asignacion, 'municipios')
data_islas, data_islas_accumul = conversion(asignacion, 'islas')
data_provincias, data_provincias_accumul = conversion(asignacion, 'provincias')

In [21]:
data_municipios_residencia, data_municipios_residencia_accumul =  conversion(residencia, 'municipios')
data_islas_residencia, data_islas_residencia_accumul = conversion(residencia, 'islas')
data_provincias_residencia, data_provincias_residencia_accumul = conversion(residencia, 'provincias')

In [22]:
print(data_islas_accumul)

fecha          index  2021/01/01  2021/01/02  2021/01/03  2021/01/04  \
0          EL HIERRO         0.0         0.0         1.0        14.0   
1      FUERTEVENTURA         1.0         5.0        17.0        21.0   
2       GRAN CANARIA        62.0       125.0       208.0       280.0   
3          LA GOMERA         0.0         1.0         1.0         5.0   
4           LA PALMA         0.0         3.0         6.0         7.0   
5          LANZAROTE        14.0        46.0        66.0        87.0   
6           TENERIFE        67.0       150.0       199.0       315.0   

fecha  2021/01/05  2021/01/06  2021/01/07  2021/01/08  2021/01/09  ...  \
0            15.0        15.0        16.0        18.0        18.0  ...   
1            47.0        63.0        78.0       107.0       137.0  ...   
2           379.0       458.0       533.0       681.0       790.0  ...   
3             5.0         5.0         5.0         8.0        13.0  ...   
4            11.0        11.0        13.0        22.0

In [23]:
data_provincias.to_excel('data/provincias_asignacion.xlsx', index=False, encoding="utf8")
data_provincias_accumul.to_excel('data/provincias_asignacion_acc.xlsx', index=False, encoding="utf8")
data_islas.to_excel('data/islas_asignacion.xlsx', index=False, encoding="utf8")
data_islas_accumul.to_excel('data/islas_asignacion_acc.xlsx', index=False, encoding="utf8")
data_municipios.to_excel('data/municipios_asignacion.xlsx', index=False, encoding="utf8")
data_municipios_accumul.to_excel('data/municipios_asignacion_acc.xlsx', index=False, encoding="utf8")

In [24]:
data_provincias_residencia.to_excel('data/provincias_residencia.xlsx', index=False, encoding="utf8")
data_provincias_residencia_accumul.to_excel('data/provincias_residencia_acc.xlsx', index=False, encoding="utf8")
data_islas_residencia.to_excel('data/islas_residencia.xlsx', index=False, encoding="utf8")
data_islas_residencia_accumul.to_excel('data/islas_residencia_acc.xlsx', index=False, encoding="utf8")
data_municipios_residencia.to_excel('data/municipios_residencia.xlsx', index=False, encoding="utf8")
data_municipios_residencia_accumul.to_excel('data/municipios_residencia_acc.xlsx', index=False, encoding="utf8")

In [25]:
df_capitales.to_csv('mapas/capitales.csv', index=False, sep=",")